In [ ]:
#### Imports 

In [18]:
#general imports
import pandas as pd
import numpy as np

#qiskit imports
from qiskit import Aer, execute
from qiskit.aqua import QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

#scikit-learn imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from qiskit_machine_learning.kernels import QuantumKernel

### Load data

In [2]:
df = pd.read_csv('ai4i2020.csv')

df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


### Feature Preparation

In [8]:
dic = {'L' : -1, 'M' : 0, 'H'  : 1}

df['TypeNum'] = df['Type'].apply(lambda x : dic[x])

In [9]:
features_labels = ['TypeNum', 'Air temperature [K]',
    'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
    'Tool wear [min]']
    
target_label = ['Machine failure']

df = df[features_labels + target_label]

### train-test split and standardize

In [10]:
sample_train, sample_test, target_train, target_test = train_test_split(
    df[features_labels], df[target_label], test_size=0.2, random_state=42)

#8000 for training, 2000 for testing

In [14]:
# Standardize
ss = StandardScaler()
sample_train = ss.fit_transform(sample_train)
sample_test = ss.transform(sample_test)

# Quantum SVM approach

In [22]:
shots = 10e03

feature_dimension = len(features_labels)
backend = Aer.get_backend('statevector_simulator')
feature_map = ZZFeatureMap(feature_dimension, reps = 2)

In [23]:
QKernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

example: parametric circuit for first two samples

In [28]:
QKernel.construct_circuit(sample_train[0], sample_train[1]).decompose().draw(output= 'mpl')

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."